In [1]:
import os
import cv2
import numpy as np
from PIL import Image as PImage
import matplotlib.pyplot as plt
np.random.seed(42)

In [2]:
# Генерация данных
imgSrc = 'DATA/CarvanaImageMasking/train/'
imgTarget = 'DATA/Stored/train/'
maskSrc = 'DATA/CarvanaImageMasking/train_masks/'
maskTarget = 'DATA/Stored/train_masks/'

def load_img(path):
    img = cv2.imread(str(path))
    img = cv2.copyMakeBorder(img, 0, 0, 1, 1, cv2.BORDER_REFLECT_101)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img.astype(np.uint8)


def load_mask(path):
    with open(path, 'rb') as f:
        with PImage.open(f) as img:
            if '.gif' in str(path):
                img = (np.asarray(img) > 0)
            else:
                img = (np.asarray(img) > 255 * 0.5)
            img = cv2.copyMakeBorder(img.astype(np.uint8), 0, 0, 1, 1, cv2.BORDER_REFLECT_101)
            return img.astype(np.float32)

def random_hue_saturation_value(img,
                                hue_shift_limit=(-180, 180),
                                sat_shift_limit=(-255, 255),
                                val_shift_limit=(-255, 255)):

    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img)
    hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
    h = cv2.add(h, hue_shift)
    sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
    s = cv2.add(s, sat_shift)
    val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
    v = cv2.add(v, val_shift)
    img = cv2.merge((h, s, v))
    img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
    return img

def grayscale_aug(img, mask):
    car_pixels = (cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB) * img).astype(np.uint8)

    gray_car = cv2.cvtColor(car_pixels, cv2.COLOR_RGB2GRAY)

    rgb_gray_car = cv2.cvtColor(gray_car, cv2.COLOR_GRAY2RGB)

    rgb_img = img.copy()
    rgb_img[rgb_gray_car > 0] = rgb_gray_car[rgb_gray_car > 0]
    return rgb_img

def rotate(img, angle):
    rows,cols = img.shape[0], img.shape[1]
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    return cv2.warpAffine(img,M,(cols,rows))

def augment(img, mask):
    if np.random.random() < 0.5:
        img = random_hue_saturation_value(img,
                                          hue_shift_limit=(-50, 50),
                                          sat_shift_limit=(-5, 5),
                                          val_shift_limit=(-15, 15))
    else:
        img = grayscale_aug(img, mask)
    
    angle = np.random.uniform(-30, 30)
    img, mask = rotate(img, angle), rotate(mask, angle)

    return img.copy(), mask.copy()
    
paths = np.stack(([i for i in os.listdir(imgSrc)], [i for i in os.listdir(maskSrc)]), axis=1)

i = 0
steps = len(paths)
for path in paths:
    img, mask = load_img(os.path.join(imgSrc, path[0])), load_mask(os.path.join(maskSrc, path[1]))
    plt.imsave(os.path.join(imgTarget, path[0][:-4]+'.jpg'), img)
    plt.imsave(os.path.join(maskTarget, path[1][:-4]+'.jpg'), mask, cmap='gray')
    
    img, mask = augment(img, mask)
    plt.imsave(os.path.join(imgTarget, path[0][:-4]+'_GEN.jpg'), img)
    plt.imsave(os.path.join(maskTarget, path[1][:-4]+'_GEN.jpg'), mask, cmap='gray')
    
    i += 1
    print('\r       \r{:.2f}%'.format(round(i/steps*100, 2)), sep='', end='')

100.00%